In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split


In [3]:
path = 'California_Fire_Incidents.xlsx'
fire = pd.read_excel(path)
columns = ['AcresBurned', 'AirTankers', 'AdminUnit', 'Counties', 'CountyIds', 'Engines', 'Latitude', 'Longitude', 'Started']

In [44]:
fire_df = fire[columns]
print(len(fire_df))
fire_df.head()

1636


,AcresBurned,AirTankers,AdminUnit,Counties,CountyIds,Engines,Latitude,Longitude,Started
0,257314.0,NaN,Stanislaus National Forest/Yosemite National Park,Tuolumne,55,NaN,37.857000,-120.086000,2013-08-17T15:25:00Z
1,30274.0,NaN,USFS Angeles National Forest/Los Angeles Count...,Los Angeles,19,NaN,34.585595,-118.423176,2013-05-30T15:28:00Z
2,27531.0,NaN,CAL FIRE Riverside Unit / San Bernardino Natio...,Riverside,33,NaN,33.709500,-116.728850,2013-07-15T13:43:00Z
3,27440.0,NaN,Tahoe National Forest,Placer,31,NaN,39.120000,-120.650000,2013-08-10T16:30:00Z
4,24251.0,NaN,Ventura County Fire/CAL FIRE,Ventura,56,117.0,0.000000,0.000000,2013-05-02T07:01:00Z


In [5]:
fire_df.isna().sum()

AcresBurned       3
AirTankers     1608
AdminUnit         0
Counties          0
CountyIds         0
Engines        1445
Latitude          0
Longitude         0
Started           0
dtype: int64

In [62]:
#fire_df = fire_df.drop(['AirTankers', 'Engines', 'AdminUnit'], axis=1)
fire_df = fire_df.dropna()
fire_df = fire_df[fire_df['Latitude'] != float(0)]
fire_df = fire_df[fire_df['Longitude'] != float(0)]
len(fire_df)
fire_df.to_csv('cleaned_fire.csv')

In [63]:
fire_df.tail()
print(len(fire_df))

1479


In [8]:
fire_df.dtypes

AcresBurned    float64
Counties        object
CountyIds       object
Latitude       float64
Longitude      float64
Started         object
dtype: object

In [49]:
months = []
years = []
for index, row in fire_df.iterrows():
   #print(date)
    month = row['Started'].split('-')[1]
    year = row['Started'].split('-')[0]
    months.append(np.float32(month))
    years.append(np.float32(year))

In [50]:
fire_df = fire_df.drop('Started', axis=1)
fire_df['Month'] = months
fire_df['Year'] = years
fire_df.head()
#fire_df.to_csv('Wildfires 2013-2019.csv')

,AcresBurned,Counties,CountyIds,Latitude,Longitude,Month,Year
0,257314.0,Tuolumne,55,37.857000,-120.086000,8.0,2013.0
1,30274.0,Los Angeles,19,34.585595,-118.423176,5.0,2013.0
2,27531.0,Riverside,33,33.709500,-116.728850,7.0,2013.0
3,27440.0,Placer,31,39.120000,-120.650000,8.0,2013.0
5,22992.0,Fresno,10,37.279000,-119.318000,7.0,2013.0


In [51]:
labels = fire_df['Counties']
features = fire_df.drop(['Counties', 'CountyIds'], axis=1)
features

,AcresBurned,Latitude,Longitude,Month,Year
0,257314.0,37.857000,-120.086000,8.0,2013.0
1,30274.0,34.585595,-118.423176,5.0,2013.0
2,27531.0,33.709500,-116.728850,7.0,2013.0
3,27440.0,39.120000,-120.650000,8.0,2013.0
5,22992.0,37.279000,-119.318000,7.0,2013.0
...,...,...,...,...,...
1628,10.0,39.423833,-121.579738,7.0,2019.0
1629,10.0,41.946220,-122.401570,6.0,2019.0
1630,10.0,39.839580,-121.957000,4.0,2019.0
1631,9.0,33.827979,-117.499619,10.0,2019.0


In [55]:
dummies = pd.get_dummies(labels)
county_names = dummies.columns
#print(county_names)
one_hot_labels = dummies.to_numpy()
scaler = MinMaxScaler()
#acres = features['AcresBurned'].to_numpy().reshape(-1,1)
scaled = scaler.fit_transform(features)
one_hot_labels[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=uint8)

In [56]:
print(features.shape)
one_hot_labels.shape

(1479, 5)


(1479, 59)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(scaled, one_hot_labels, test_size=0.2)

## Rough draft model


In [20]:
import sys
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
# from tensorflow import keras
from tensorflow.keras import layers, Sequential, Input
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, concatenate
# from tensorflow.keras.layers.core import *
from tensorflow.keras.models import Model
from tensorflow.keras.layers.experimental import preprocessing
# from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input


def build_dnn():

    
    normalizer = layers.BatchNormalization(momentum=0.98, epsilon=.001, trainable=True)

    model = Sequential([

        #normalizer,
        layers.Dense(24, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(1)
    ])
    model = Model([img_input, num_input], out)
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0001), metrics=['accuracy'])
    print("Model compiled")
    return model

In [21]:
def build_dnn(input_shape, numclasses):
    feature_input = Input(shape=input_shape)
    model = Dense(8, activation='relu')(feature_input)
    model = Dense(4, activation='relu')(model)
    model = Dense(1, activation='relu')(model)
    output = Dense(numclasses, activation='sigmoid')(model)
    model = Model(inputs=feature_input, outputs=output)
    print(model.summary())
    return(model)

In [64]:
def merged_build_dnn(fire_shape, climate_shape, numclasses):
    fire_input = Input(shape=fire_shape)
    model1 = Dense(64, activation='relu')(fire_input)
    model1 = Dense(16, activation='relu')(model1)
    #model1 = Dense(4, activation='relu')(model1)
    
    output1 = Dense(4, activation='relu')(model1)
    
    climate_input = Input(shape=climate_shape)
    model2 = Dense(64, activation='relu')(climate_input)
    model2 = Dense(16, activation='relu')(model2)
    output2 = Dense(4, activation='relu')(model2)
    
    merged_model = concatenate([output1, output2])
    
    dp1 = Dense(4, activation='relu')(merged_model)
    output = Dense(1, activation='sigmoid')(dp1)

    model = Model(inputs=[fire_input, climate_input], outputs=output)
    print(model.summary())
    return(model)

In [22]:
model = build_dnn((5,), len(one_hot_labels[0]))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense (Dense)                (None, 8)                 48        
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
_________________________________________________________________
dense_3 (Dense)              (None, 59)                118       
Total params: 207
Trainable params: 207
Non-trainable params: 0
_________________________________________________________________
None


In [59]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0001), metrics=['accuracy'])

model.fit(X_train, y_train, validation_split=0.2, epochs=10)
model.evaluate(X_test, y_test, verbose=0)


Epoch 1/10
30/30 [==============================] - 0s 4ms/step - loss: 4.0702 - accuracy: 0.0941 - val_loss: 4.0707 - val_accuracy: 0.0675
Epoch 2/10
30/30 [==============================] - 0s 3ms/step - loss: 4.0696 - accuracy: 0.0941 - val_loss: 4.0701 - val_accuracy: 0.0675
Epoch 3/10
30/30 [==============================] - 0s 5ms/step - loss: 4.0689 - accuracy: 0.0941 - val_loss: 4.0696 - val_accuracy: 0.0675
Epoch 4/10
30/30 [==============================] - 0s 5ms/step - loss: 4.0682 - accuracy: 0.0941 - val_loss: 4.0690 - val_accuracy: 0.0675
Epoch 5/10
30/30 [==============================] - 0s 2ms/step - loss: 4.0676 - accuracy: 0.0941 - val_loss: 4.0684 - val_accuracy: 0.0675
Epoch 6/10
30/30 [==============================] - 0s 3ms/step - loss: 4.0669 - accuracy: 0.0941 - val_loss: 4.0679 - val_accuracy: 0.0675
Epoch 7/10
30/30 [==============================] - 0s 3ms/step - loss: 4.0662 - accuracy: 0.0941 - val_loss: 4.0673 - val_accuracy: 0.0675
Epoch 8/10
30/30 [==

[4.065078258514404, 0.08445946127176285]

In [60]:
truth = y_test.argmax(axis=1) #.argmax(axis=1)

def get_names(data):
    county_labels = []
    for i in np.arange(len(data)):
        county = county_names[data[i]]
        county_labels.append(county)
    return county_labels
#print(county_labels)
truth_labels = get_names(truth)

predic = model.predict(X_test)
predic = predic.argmax(axis=1)
predic_labels = get_names(predic)
print(truth_labels)
print(predic_labels)

['Shasta', 'Lake', 'Yuba', 'San Luis Obispo', 'Tulare', 'Alameda', 'Butte', 'Butte', 'Stanislaus', 'Santa Clara', 'Placer', 'Shasta', 'Modoc', 'Mendocino', 'Alameda', 'San Bernardino', 'Riverside', 'Fresno', 'San Diego', 'Madera', 'San Diego', 'Riverside', 'Los Angeles', 'Mariposa', 'Alameda', 'Napa', 'Humboldt', 'Kern', 'Madera', 'Stanislaus', 'Kern', 'Mariposa', 'Riverside', 'Solano', 'Los Angeles', 'Contra Costa', 'Riverside', 'Trinity', 'Tuolumne', 'Yuba', 'San Diego', 'Plumas', 'Tehama', 'Tulare', 'Yuba', 'Fresno', 'Calaveras', 'Riverside', 'San Bernardino', 'El Dorado', 'Riverside', 'Riverside', 'Lassen', 'San Benito', 'Riverside', 'Riverside', 'Merced', 'Merced', 'Mono', 'Monterey', 'Kern', 'Butte', 'Butte', 'Calaveras', 'Tuolumne', 'Tuolumne', 'Siskiyou', 'Kern', 'Mendocino', 'Modoc', 'Santa Clara', 'Amador', 'Siskiyou', 'Santa Clara', 'San Luis Obispo', 'Madera', 'Modoc', 'Monterey', 'Kern', 'San Bernardino', 'Shasta', 'Siskiyou', 'San Bernardino', 'Humboldt', 'Ventura', 'Lass

In [34]:
county_names[0]

'Alameda'